In [1]:
!pip install findspark

In [2]:
!pip install pyarrow

In [3]:
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SparkContext
from lib.prepro import Preprocessing, SparkDataFrame
from tqdm.notebook import tqdm
import dask.dataframe as dd
import os
from pprint import pprint
import pandas as pd

In [4]:
# PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터/*'
# PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/*'
# PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터/*'

PATH_CCTV_DATA = './cctv_data/*'
PATH_SAND_DATA = './sand_data/*'
PATH_SAND_LABEL = './sand_label_data/*'

# PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터'
# PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터'
# PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터'

In [5]:
prepro = Preprocessing()
sdf = SparkDataFrame()

In [6]:
# extract file path 
cctv_paths = prepro.get_all_file_paths(PATH_CCTV_DATA)
sand_paths = prepro.get_all_file_paths(PATH_SAND_DATA)
sand_label_paths = prepro.get_all_file_paths(PATH_SAND_LABEL)

In [7]:
# sand_paths = prepro.get_all_file_paths(PATH_SAND_DATA)

In [8]:
# make dataframe
cctv_df = sdf.get_spark_dataframe(cctv_paths)
sand_df = sdf.get_spark_dataframe(sand_paths)
sand_label_df = sdf.get_spark_dataframe(sand_label_paths)

### convert dataframe to parquet

In [11]:
# parquet_path = './preprocessed_data/sand.parquet'
# parquet_name = 'sand2.parquet.gzip'
# # full_parquet_path = parquet_path + parquet_name
# full_parquet_path = os.path.join(parquet_path, parquet_name)
# os.path.exists(parquet_path)

In [12]:
# parquet_data = limited_ten_sand_df.toPandas()

In [13]:
# parquet_data.to_parquet(full_parquet_path, engine='pyarrow')
# parquet_data.to_parquet(full_parquet_path, engine='pyarrow', compression='gzip', index=False)

In [14]:
# test_parquet = pd.read_parquet(full_parquet_path)

### Save database

In [15]:
# import sqlite3
# conn = sqlite3.connect('test.db')
# cursor = conn.cursor()

In [9]:
!pip install psycopg2-binary

In [10]:
import psycopg2
from psycopg2 import sql

conn = psycopg2.connect(
    dbname="postgres",  # 초기 연결은 일반적으로 기본 데이터베이스로
    user="postgres",
    password="1234",
    host="postgres",
    port="5432"
)
conn.autocommit = True  # 데이터베이스 생성 시 autocommit 설정

# 커서 생성
cursor = conn.cursor()

In [11]:


# 데이터베이스 생성 쿼리 실행
# database_name = "testdb"
# cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(database_name)))

In [12]:
# 테이블 생성 SQL 명령어
create_table_query = sql.SQL("""
CREATE TABLE IF NOT EXISTS file_data (
    full_path TEXT NOT NULL,
    file_id VARCHAR(20) PRIMARY KEY,
    file_name VARCHAR(255) NOT NULL,
    folder_name VARCHAR(255) NOT NULL,
    file_size BIGINT NOT NULL
)
""")

cursor.execute(create_table_query)

# 변경 사항 커밋
conn.commit()

In [ ]:
%%time
url = "jdbc:postgresql://postgres:5432/postgres"

properties = {
    "user": "postgres",
    "password": "1234",
    "driver": "org.postgresql.Driver"
}

# DataFrame을 PostgreSQL에 저장
sand_df.write \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", "file_data") \
    .option("batchsize", "5") \
    .option("numPartitions", "10") \
    .mode("overwrite") \
    .options(**properties) \
    .save()

In [ ]:
cursor.execute("SELECT * FROM file_data")

In [ ]:
cursor.fetchall()

### Test

In [34]:
df = pd.read_parquet(full_parquet_path)

In [49]:
try:
    df.to_sql('parquet_data', connection, if_exists='replace', index=False)
    connection.commit()
except Exception as e:
    print("SQL에 쓰는 중 오류 발생:", e)

In [ ]:
cursor.execute("SELECT * FROM file_data WHERE type='table';")
tables = cursor.fetchall()
print(tables)  # Check if 'parquet_data' is listed


In [51]:
try:
    cursor.execute('SELECT * FROM parquet_data')
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except sqlite3.OperationalError as e:
    print("OperationalError:", e)

('./sand_data/TS_1.모래입자크기분류_1/0010067.png', '0010067', '0010067.png', 'TS_1.모래입자크기분류_1', 3018192)
('./sand_data/TS_1.모래입자크기분류_1/0010049.png', '0010049', '0010049.png', 'TS_1.모래입자크기분류_1', 3287386)
('./sand_data/TS_1.모래입자크기분류_1/0010021.png', '0010021', '0010021.png', 'TS_1.모래입자크기분류_1', 3286182)
('./sand_data/TS_1.모래입자크기분류_1/0010004.png', '0010004', '0010004.png', 'TS_1.모래입자크기분류_1', 2930369)
('./sand_data/TS_1.모래입자크기분류_1/0010094.png', '0010094', '0010094.png', 'TS_1.모래입자크기분류_1', 3266231)
('./sand_data/TS_1.모래입자크기분류_1/0010005.png', '0010005', '0010005.png', 'TS_1.모래입자크기분류_1', 3293061)
('./sand_data/TS_1.모래입자크기분류_1/0010006.png', '0010006', '0010006.png', 'TS_1.모래입자크기분류_1', 3143184)
('./sand_data/TS_1.모래입자크기분류_1/0010057.png', '0010057', '0010057.png', 'TS_1.모래입자크기분류_1', 3275718)
('./sand_data/TS_1.모래입자크기분류_1/0010015.png', '0010015', '0010015.png', 'TS_1.모래입자크기분류_1', 3023326)
('./sand_data/TS_1.모래입자크기분류_1/0010047.png', '0010047', '0010047.png', 'TS_1.모래입자크기분류_1', 3017624)


In [11]:
# read json
json_data = sdf.get_spark_json(sand_label_paths)

In [12]:
prepro.get_folder_name(cctv_paths[0])

'4.계기용변압기(PT)'

In [8]:
pprint(cctv_data_info)
pprint(sand_data_info)
pprint(sand_label_data_info)

('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 1457336 \n'
 '            png counts: 0 \n'
 '            jpg counts: 1093002 \n'
 '            csv counts: 364334\n'
 '            json counts: 0\n'
 '            etc counts: 0 \n'
 '            -------------------------------------- \n'
 '            ')
('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 490755 \n'
 '            png counts: 490755 \n'
 '            jpg counts: 0 \n'
 '            csv counts: 0\n'
 '            json counts: 0\n'
 '            etc counts: 0 \n'
 '            -------------------------------------- \n'
 '            ')
('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 490755 \n'
 '            png counts: 0 \n'
 '            jpg counts: 0 \n'
 '            csv counts: 0\n'
 '            json counts: 49